---
# Demo Use Case - Object Detection and Video Player (Part 2)

The raw video is now being continuously ingested into SDP.

As each video frame is ingested, the object detection model is used to detect objects such as cars, buses, and people.
The object detection model runs as a Flink job in SDP. It uses the popular YOLO (You Only Look Once) object detection model.
The Flink job uses Java bindings for TensorFlow and it uses GPUs to performance high speed deep learning inference.

We use a cluster of 6 servers, each with an NVIDIA Tesla V100 GPU. These are used in parallel by our Flink job.

See the [SDP UI](https://videodemo1.nautilus-lab-sulfur.com/).

Now the data scientist wants to view the result of the object detection model in real-time.
This is shown as a sequence of video frames with labeled boxes around detected objects.
The images from all cameras is combined into the same SDP stream.
Each image is tagged with the original camera number.

---

### Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import VideoPlayer
import os

### Define Pravega connection parameters

In [ ]:
#gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
#gateway = '10.246.27.131:54672'
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
scope = 'examples'
#stream = 'object-detector-input-video'
#stream = 'object-detector-output-video'
#stream = 'generated-video'
#stream = 'object-detector-input-grid-video'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### About the timestamp index
This is an index from timestamp to begin stream cut, end stream cut, and event pointer.
This index is maintained by the job in [index_builder.ipynb](index_builder.ipynb).
The video player uses the event pointer to fetch an individual frame. It uses the end stream cut to play from a particular frame.

### Video Player

In [ ]:
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoPlayer
imp.reload(video);

In [5]:
player = VideoPlayer(pravega_client=pravega_client, scope=scope)
player.interact()